# Perceptron Training


The task is to build a feature-based multi-class perceptron classifier (see separate handout on the perceptron algorithm) and evaluate its accuracy.

You are given a *training set* and a *test set* (restaurant_train.txt and restaurant_test.txt) that on each line lists 
1. the class, and 
2. a list of feature-value pairs. 

Each line in both files looks as follows:
    class f1:v1f2:v2... fn:vn
 

- Each line is an instance (one single example).
- The class is an integer {0,1,2}, i.e. this is a three-class problem with the classes numbered 0-2.
- The feature list is sparse in the sense that features with value 0 are not mentioned separately. All feature values are either 0 or 1.

Here is an example from restaurant_train.txt:
    0 2:1 10:1 26:1 33:1 34:1 57:1 97:1 100:1
 
This example then belongs to class 0, its features {2,10,26,33,34,57,97,100} are all 1 while the remaining features have value 0.

The data set is a set of restaurant reviews and the labels indicate whether it is a review about Italian, Chinese or Mexican food.  We don't need to worry about what the feature numbers correspond to: in this homework the task is to focus on the perceptron algorithm and feature representations. The idea is to just implement the algorithm without concerning ourselves about how the reviews were turned into features. Note that you need to separately figure out how long the feature vector should be.

In short, the task is to:
- read in the data
- convert each example to a vector (a list) representation in Python 
- implement the classifier using vector operations, and 
- evaluate the performance.

In [1]:
# Create a function that takes a filename and returns a list of tuples. 
#[(class, [features present/absent]), (class, [features present/absent]), etc]

#listTuples = []

# Figure out the number of features by reading in ALL the data (training, dec, test) 
# and looking for the largest number in the (feature:value) pairs. Add one for the prior.

def filetolist(filename, inputList):
    
    for line in open(filename):
        # features = [0] * (max_feature_num + 1)
        features = [0] * 9492 #creates list of size 9492 with all 0s
        features[-1] = 1 # this is the prior, value = 1
        l = line.split() #list of items ['class', 'feature:value', 'feature:value']
        c = l[0] # just the class: 0, 1, or 2
        for element in l[1:]: #for all the feature:value pairs
            x = element.split(':') 
            feature_present = int(x[0]) #the feature num
            # max_feature_num = 0
            # if int(feature_present) > max_feature_num:
            # # max_feature_num = int(feature_present)
            features[feature_present] = 1 #Index features[] using the feature num and change that value to a 1
        inputList.append((c, features)) #creat tuple (class num, features list) and add to larger list
    #print(inputList[0], end = ' ')
        
#filetolist('restaurant_train.txt', listTuples)
#filetolist('restaurant_test.txt')

In [2]:
# Create functions dotproduct(), vec_sub(), and vec_add(), all of which take two lists (vectors) 
# as arguments, and calculate 1) the dot product of the two vectors, 2) subtract one vector from another, 
# and 3) add one vector to another. 

def dotproduct(vector1, vector2):
    dproduct = 0
    for item1, item2 in zip(vector1, vector2):
        dproduct += item1 * item2
    return dproduct

#test
v1 = [1, 2, 3]
v2 = [4, 5, 6]
# 4 + 10 + 18 = 32

dotproduct(v1, v2)

32

In [3]:
def vec_add(vector1, vector2):
    sumVectors = []
    for item1, item2 in zip(vector1, vector2):
        sumVectors.append(item1 + item2)
    return sumVectors

#test
v1 = [1, 2, 3]
v2 = [4, 5, 6]
# [5,7,9]

vec_add(v1, v2)

[5, 7, 9]

In [4]:
def vec_sub(vector1, vector2):
    difVectors = []
    for item1, item2 in zip(vector1, vector2):
        difVectors.append(item1 - item2)
    return difVectors

#test
v1 = [1, 2, 3]
v2 = [4, 5, 6]
# [-3,-3,-3]

vec_sub(v1, v2)

[-3, -3, -3]

Step 1: Perceptron Training
For 3 classes, I need 3 weights vectors: {w1, w2, w3}, all initialized to 0.
There are 9492 features plus the prior for a list of length 9493.

1. Read in the training data that has 1000 lines using filetolist()
2. Classify each line (restaurant) as class 0, 1, or 2 by taking the Dot Product of the 3 weights vectors individually and the current restaurant's features vector.
3. If my model classifies the restaurant correctly, move to the next line. 
4. If my model classifies the restaurant incorrectly, I should use the vec_add() function to increase the weights for the correct class and vec_sub() to decrease the weights for wrongly-guessed class according to the current restaurant's features. I should leave the 3rd class alone.
5. Keep track of how many misclassifications my model makes. If that number reaches 0 (all restaurants were guessed correctly), I can stop my model.

In [5]:
# weights vectors
w0 = [0] * 9492
w1 = [0] * 9492
w2 = [0] * 9492

trainingData = [] # This will be size 1000 for 1000 restaurants
filetolist('restaurant_train.txt', trainingData)
#print(trainingData[0])
#len(trainingData)

def trainWeights(trainingData, w0, w1, w2):

    noErrors = False
    while(noErrors == False):      
        numErrors = 0

        for item in trainingData: #for every ('class', [features vector]) tuple...

            if item[0] == '0':
                actualClassID = '0'
            elif item[0] == '1':
                actualClassID = '1'
            else:
                actualClassID = '2'

            dp_class0 = dotproduct(w0, item[1])
            dp_class1 = dotproduct(w1, item[1])
            dp_class2 = dotproduct(w2, item[1])
            winner = max(dp_class0, dp_class1, dp_class2)
            
            if winner == dp_class0:
                winnerID = '0'
            elif winner == dp_class1:
                winnerID = '1'
            else:
                winnerID = '2'

            if winnerID == '0' and actualClassID != '0': #incorrect guess
                numErrors += 1
                w0 = vec_sub(w0, item[1])             
                if(actualClassID == '1'):
                    w1 = vec_add(w1, item[1])
                elif(actualClassID == '2'):
                    w2 = vec_add(w2, item[1])

            elif winnerID == '1' and actualClassID != '1': #incorrect guess
                numErrors += 1
                w1 = vec_sub(w1, item[1])             
                if(actualClassID == '0'):
                    w0 = vec_add(w0, item[1])
                elif(actualClassID == '2'):
                    w2 = vec_add(w2, item[1])

            elif winnerID == '2' and actualClassID != '2': #incorrect guess
                numErrors += 1
                w2 = vec_sub(w2, item[1])             
                if(actualClassID == '1'):
                    w1 = vec_add(w1, item[1])
                elif(actualClassID == '0'):
                    w0 = vec_add(w0, item[1])

            #print('Winning class is:', winnerID)
            #print('Actual class is: ', actualClassID)
            #print('w0:', w0[0:10]) 
            #print('w1:', w1[0:10])
            #print('w2:', w2[0:10])

        print('Errors: ', numErrors)
        if(numErrors == 0):
            noErrors = True
    return w0, w1, w2

w0, w1, w2 = trainWeights(trainingData, w0, w1, w2)

Errors:  330
Errors:  88
Errors:  58
Errors:  34
Errors:  20
Errors:  11
Errors:  9
Errors:  2
Errors:  0


I now have a function that trains my weight vectors for each of my three restaurant classes.
Now, if I wanted to, I could figure out HOW LONG I should train my weight vectors for better accuracy. For this, I would use my development data set.

1. Read in restaurant_dev.txt.
2. Run trainWeights() on Dev set.
3. Count how many classifications were correct out of 250.
4. If the subsequent classifications are LESS ACCURATE than the previous, then stop training.

In [6]:
# dev set has 250 restaurants on which to test different weights
devData = []
filetolist('restaurant_dev.txt', devData)


After trying out different weights and deciding what the best time to stop training is (optional), it's time to evaluate my test set.

1. Read in restaurant_test.txt
2. Run classifier and count how many errors are made out of 500

In [7]:
#print('Weights after training:')
#print('w0:', w0[0:15])
#print('w1:', w1[0:15])
#print('w2:', w2[0:15])

# there are 500 resturants in the test set
testData = []
filetolist('restaurant_test.txt', testData)

#print('test data:', testData[0])

def testModel(testData, w0, w1, w2):
    numErrors = 0
    numCorrect = 0
    
    for item in testData:
        if item[0] == '0':
            actualClassID = '0'
        elif item[0] == '1':
            actualClassID = '1'
        else:
            actualClassID = '2'
    
        dp_class0 = dotproduct(w0, item[1])
        dp_class1 = dotproduct(w1, item[1])
        dp_class2 = dotproduct(w2, item[1])
        winner = max(dp_class0, dp_class1, dp_class2)
        
        if winner == dp_class0:
            winnerID = '0'
        elif winner == dp_class1:
            winnerID = '1'
        else:
            winnerID = '2'

        if winnerID == '0' and actualClassID != '0': #incorrect guess
            numErrors += 1
        elif winnerID == '1' and actualClassID != '1': #incorrect guess
            numErrors += 1
        elif winnerID == '2' and actualClassID != '2': #incorrect guess
            numErrors += 1
        else:
            numCorrect += 1
    
    #print('Correct:', numCorrect)
    print('Errors:', numErrors)
    accuracy = (500 - numErrors) * 100 / 500
    return accuracy

accuracy = testModel(testData, w0, w1, w2)
print('Accuracy:', accuracy)

Errors: 56
Accuracy: 88.8
